# **LoopsResultsExploration**

## **1. Importing and First Proccesing**

In [11]:
import pandas as pd
from DataCleaning import *

pd.options.display.max_columns = 50

In [12]:
raw_data = pd.read_excel(cleaning_config['raw_data_path'])
raw_data = clean_data(raw_data)

In [13]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5071 entries, 0 to 5070
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   5071 non-null   int64 
 1   subject                 5071 non-null   object
 2   step_num                5071 non-null   int64 
 3   uid                     5071 non-null   int64 
 4   trial_start_time        5071 non-null   int64 
 5   rt                      5071 non-null   int32 
 6   response                5071 non-null   object
 7   loop_step               5071 non-null   int32 
 8   trial_order             5071 non-null   int64 
 9   var_mapping             5071 non-null   object
 10  trial_set               5071 non-null   int64 
 11  trial                   5071 non-null   int64 
 12  core_program            5071 non-null   int64 
 13  variant                 5071 non-null   object
 14  step_id                 5071 non-null   int64 
 15  text

In [14]:
raw_data.head()

,index,subject,step_num,uid,trial_start_time,rt,response,loop_step,trial_order,var_mapping,trial_set,trial,core_program,variant,step_id,text1,response_needed,expected_response,is_loop,loop_type,loop_type_switch,n_iterations,n_loop_lines,step_type,prev_loop_type,correct,same_as_prev_loop_type
0,7,107B,7,30,38002,818,2,1,1,1=a;2=i,2,4,2,FWWWWFFW,3,while a >= 2:\n a /= 2,True,"[4, 2, 1]",True,while,True,3,1,loop,0,False,False
1,8,107B,8,30,38811,302,1,2,1,1=a;2=i,2,4,2,FWWWWFFW,3,while a >= 2:\n a /= 2,True,"[4, 2, 1]",True,while,True,3,1,loop,0,False,False
2,11,107B,11,31,43376,69,5,1,1,1=a;2=i,2,4,2,FWWWWFFW,4,while a <= 7:\n a += 2,True,"[3, 5, 7, 9]",True,while,False,4,1,loop,0,False,False
3,12,107B,12,31,44123,237,7,2,1,1=a;2=i,2,4,2,FWWWWFFW,4,while a <= 7:\n a += 2,True,"[3, 5, 7, 9]",True,while,False,4,1,loop,0,False,False
4,13,107B,13,31,44963,327,9,3,1,1=a;2=i,2,4,2,FWWWWFFW,4,while a <= 7:\n a += 2,True,"[3, 5, 7, 9]",True,while,False,4,1,loop,0,False,False


## **2. Exploring Outliers**

In [15]:
def is_outlier(x, x_q1, x_q3, x_iqr, threshold):
    return (x_q1 - x) / x_iqr >= threshold or (x - x_q3) / x_iqr >= threshold

### 2.1. Response-Time Between Subjects

In [16]:
# filtering only necessary columns
response_times = raw_data[['subject', 'step_num', 'rt']].copy()
response_times.drop_duplicates(inplace=True)

# calculating quantiles
g_rt_q1, g_rt_q3 = response_times['rt'].quantile([0.25, 0.75])
g_rt_iqr = g_rt_q3 - g_rt_q1

In [20]:
# checking for outliers in terms of median response time
med_rt_per_subject = response_times[['rt', 'subject']].groupby('subject').median()
med_rt_per_subject.columns = ['med_rt']

med_outlier_mask = med_rt_per_subject['med_rt'].apply(is_outlier, args=(g_rt_q1, g_rt_q3, g_rt_iqr, 1.5))
med_outliers = med_rt_per_subject[med_outlier_mask].index

if med_outliers.size > 0:
    print(f'Seems that: {list(med_outliers)} are outliers in terms of mean response time within subject.')
else:
    print("No Outliers detected! at least in terms of median response time within subject.")

No Outliers detected! at least in terms of median response time within subject.


In [ ]:
# checking for outliers in terms of response time variance (standard deviation)
rt_std_per_subject = response_times.groupby('subject').std()
rt_std_per_subject.columns = ['rt_std']

std_q3, std_q1 = rt_std_per_subject['rt_std'].quantile([0.75, 0.25])
std_iqr = std_q3 - std_q1

std_outlier_mask = rt_std_per_subject['rt_std'].apply(is_outlier, args=(std_q1, std_q3, std_iqr, 1.5))
std_outliers = rt_std_per_subject[std_outlier_mask].index
if std_outliers.size > 0:
    print(f'Seems that: {list(std_outliers)} are outliers in terms of response time variance within subject.')
else:
    print("No Outliers detected! at least in terms of response time variance within subject.")

Seems that: ['103B'] are outliers in terms of response time variance within subject.


In [ ]:
print(f'General response time std: {g_rt_std}')
rt_std_per_subject.loc[['103B', '108B']]

General response time std: 1838.0058363095263


,rt_std
subject,
103B,2988.853086
108B,2774.760539


#### *It seems that in terms of response time variance within subject, **103B** is an outlier, and **108B** is arguably also an outlier.*

In [ ]:
print(f"103B's grade: {(response_time_agg.loc['103B', 'rt_std'] - std_q3) / std_iqr}")
print(f"108B's grade: {(response_time_agg.loc['108B', 'rt_std'] - std_q3) / std_iqr}")

103B's grade: 1.856285494638352
108B's grade: 1.470637240522526


#### *I would not filter 103B and 108B out, because they are not so extreme.*

### 2.2. Mistakes Rate (%) Between Subjects

### 2.3. Program Mistakes Rate (%) Within Subject 

### 2.3. Single Steps Response-Time Within Subject

In [ ]:
# calculating response time quantiles and IQR per subject
quantiles_per_subject = response_times[['rt', 'subject']].groupby('subject').quantile([0.25, 0.75]).unstack()
quantiles_per_subject.columns = ['q1', 'q3']
quantiles_per_subject['iqr'] = quantiles_per_subject['q3'] - quantiles_per_subject['q1']

quantiles_per_subject.head(3)

,q1,q3,iqr
subject,,,
101B,1207.0,2550.50,1343.50
102A,1245.0,3497.75,2252.75
102B,1045.0,2832.50,1787.50


In [ ]:
# finding if a step is an outlier in terms of response time within subject
def is_subjective_outlier(step):
    rt = step['rt']
    subject = step['subject']
        
    subject_quantiles = quantiles_per_subject.loc[subject]
    q1, q3, iqr = subject_quantiles['q1'], subject_quantiles['q3'], subject_quantiles['iqr']
    return is_outlier(rt, q1, q3, iqr, 2)

In [ ]:
# filtering only outliers
subjective_outlier_mask = response_times[['subject', 'rt', 'step_num']].apply(is_subjective_outlier, axis=1)
outlier_steps = response_times[subjective_outlier_mask]

KeyError: "['step_num'] not in index"

In [ ]:
outlier_steps = outlier_steps.merge(quantiles_per_subject, how='left', left_on='subject', right_index=True)
outlier_steps.head()

NameError: name 'outlier_steps' is not defined